### Machine Learning / Asunciones - Pair Programming

El objetivo de este pairprgramming es que evaluéis si vuestro set de datos cumple todas las asunciones que se deben cumplir para hacer una regresión lineal. Recordamos que estas asunciones son:
- Normalidad (ya la deberíais haber evaluado).
- Homogeneidad de varianzas.
- Independencia de las variables.

Cada asunción la deberéis testar de forma visual y analítica.

In [31]:
!pip install researchpy


In [32]:
!pip install scipy


In [33]:
import numpy as np
import pandas as pd
import random 

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import researchpy as rp
from scipy.stats import levene

plt.rcParams["figure.figsize"] = (10,8) 

In [34]:
df = pd.read_csv("data/adult.data_limpio.csv", index_col = 0)

- Asunción de normalidad:

In [35]:
df.shape

(32560, 15)

In [36]:
df.head()

,state-gov,final_weight,bachelors,education_num,marital_status,ocupation,relationship,ethnicity,gender,capital_gain,capital_lost,hours_week,country,salary,census
39,,,,,,,,,,,,,,,
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,Bajo
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,Bajo
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,Bajo
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,Bajo
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,Bajo


Observamos que nuestro DataFrame tiene más de 5 mil líneas, por lo tanto vamos a usar el test de Kolmogórov-Smirnov.

In [37]:
kstest(df["final_weight"], 'norm')

KstestResult(statistic=1.0, pvalue=0.0)

Un p-valor menor a 0.05 nos indica que debemos rechazar la hipótesis nula y que nuestros datos no son normales.

Por lo tanto, para poder introducirlo en un modelo de machine learning, deberíamos realizar una serie de cambios que veremos próximamente.

- Homogeneidad de varianzas.

Vamos a utilizar el test de Levene porque es más robusto, y se recomienda su uso para datos que no son normales.

In [38]:
resultados = {}

numericas_col = df.select_dtypes(include = np.number).drop("education_num", axis = 1).columns

for col in numericas_col:

    statistic, p_val = levene(df[col], df.education_num, center='median')
    
    resultados[col] = p_val


In [39]:
resultados

{'final_weight': 0.0,
 'capital_gain': 1.858143628044881e-151,
 'capital_lost': 1.1372411e-316,
 'hours_week': 0.0}

Como podemos observar, todos los valores son menores de 0.05, por lo tanto no se cumple la asunción. Esto quiere decir que las variables son independientes unas de otras.

Quizá nos dé estos resultados porque deberíamos estar trabajando sobre los residuos, cosa que aún no sabemos lo que es.

- Independencia de las variables.

In [40]:
crosstab, test_results, expected = rp.crosstab(df["final_weight"], df["hours_week"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

In [41]:
crosstab.head()

hours_week                                               ...  \
hours_week            1    2    3    4    5    6    7    8    9   10  ...   
final_weight                                                          ...   
12285               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
13769               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
14878               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
18827               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
19214               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

                                                                
hours_week     90   91   92   94   95   96   97   98   99  All  
final_weight                                                    
12285         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
13769         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
14878         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
18827         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
19214         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 95 columns]

A la vista de los resultados anteriores, no nos extraña que continúe dándonos 0. Estas dos variables no están correlacionadas porque los datos que corresponden a final_weight siguen una serie de... (continuará). [parámetros que reciben ¿?]

In [42]:
crosstab, test_results, expected = rp.crosstab(df["education_num"], df["hours_week"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

In [43]:
crosstab.head()

hours_week                                                       \
hours_week             1     2     3     4     5     6    7     8     9    10   
education_num                                                                   
1                    0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.00  0.01   
2                    0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.00  0.00   
3                    0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00  0.00  0.01   
4                    0.0  0.01  0.01  0.01  0.01  0.00  0.0  0.01  0.00  0.06   
5                    0.0  0.00  0.00  0.00  0.01  0.01  0.0  0.00  0.01  0.01   

               ...                                                      
hours_week     ...   90   91   92   94   95   96   97   98    99   All  
education_num  ...                                                      
1              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.16  
2              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.52  
3              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  1.02  
4              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.02  1.98  
5              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  1.58  

[5 rows x 95 columns]

Sin embargo, algunos de estos resltados sí que parecen ser algo dependientes. Tiene sentido, dado que es normal que la cantidad de horas que trabajes tenga relacioń con el número de años que has estudiado.